In [108]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy

In [109]:
model = joblib.load("../model/model_random_forest.pkl")
df = pd.read_csv("../data/merged_dataset.csv")

In [110]:
latest = df.drop(columns=['signal', 'close','future_close','price_change','datetime']).iloc[-1:]
prediction = model.predict(latest)
print(f"Prediction: {prediction[0]}")

Prediction: 1


In [111]:
input_data = np.array([[
1714122000000,0.74207383,0.59931915,346.08867559,469.73273161,304.87871676,140862.78339653,9080647727.890017,64583.2,64337.6,64410.0,259.01148043,1.2381926921696191,1.0038173633924203,0.9643008378391262]])
prediction = model.predict(input_data)
print("Predicted signal:", prediction[0])

Predicted signal: -1


C:\Users\arfan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [115]:

df['start_time'] = pd.to_datetime(df['datetime'])
df.set_index('start_time', inplace=True)

bt_df = df[['open', 'high', 'low', 'close', 'volume', 'signal']].copy()
bt_df['signal'] = bt_df['signal'].shift(-1)

bt_df.index = pd.to_datetime(bt_df.index, unit='ms')
# print(bt_df['signal'].value_counts())


bt_df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)

class SignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        signal = self.data.signal[-1]
        # print(f"TIME: {self.data.index[-1]}, Signal: {signal}, Position: {self.position}, Equity: {self.equity}")
        
        size = int(self.equity/self.data.Close[-1]*0.1)

        if signal == 1:
            if self.position.is_short:
                self.position.close()
            if not self.position:
                self.buy()

        elif signal == -1:
            if self.position.is_long:
                self.position.close()
            if not self.position:
                self.sell()


# Run backtest
bt = Backtest(bt_df, SignalStrategy, cash=1000000, commission=.002)
result = bt.run()
print(result)
bt.plot()


Start                     2024-02-21 06:00:00
End                       2025-04-12 05:00:00
Duration                    415 days 23:00:00
Exposure Time [%]                    93.72997
Equity Final [$]                  564977.3848
Equity Peak [$]                  1564394.4544
Commissions [$]                 10597825.4152
Return [%]                          -43.50226
Buy & Hold Return [%]                61.04576
Return (Ann.) [%]                   -38.36499
Volatility (Ann.) [%]                31.75099
CAGR [%]                            -39.40883
Sharpe Ratio                         -1.20831
Sortino Ratio                        -1.26338
Calmar Ratio                         -0.57373
Alpha [%]                           -46.06429
Beta                                  0.04197
Max. Drawdown [%]                   -66.86934
Avg. Drawdown [%]                    -3.16105
Max. Drawdown Duration      358 days 02:00:00
Avg. Drawdown Duration        9 days 06:00:00
# Trades                          

GridPlot(id='p8673', ...)